<a href="https://colab.research.google.com/github/JunEden/Tensorflow/blob/main/%E3%80%8C%E5%9C%96%E5%BD%A2%E5%91%88%E7%8F%BE_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [ ]:
ori_imgs = []
for p in x_test:
    img = Image.open(p).resize((299, 299)).convert("RGB") # resize大小依模型修改
    ori_imgs.append(np.array(img))
ori_imgs = np.array(ori_imgs)
preprocess_imgs = preprocess_input(ori_imgs)

In [ ]:
model.load_weights('')

pre = model.predict(preprocess_imgs).argmax(axis=1)
print(classification_report(y_test, preX))

In [ ]:
# 計算『混淆矩陣』(Confusion Matrix)，顯示測試集分類的正確及錯認總和數
 
pd.crosstab(y_test, pre, rownames=['實際值'], colnames=['預測值'])

# ROC 曲線與 auc

In [ ]:
pre_ori = model.predict(preprocess_imgs)
pre = pre_ori[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, pre)
plt.plot(fpr, tpr)

In [ ]:
plt.plot(fpr, tpr, label='ROC curve')

plt.plot([0, 1], [0, 1], 'k--')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
auc1 = auc(fpr, tpr)
## Plot the result
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, color = 'orange', label = 'AUC = %0.2f' % auc1)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show() 